# Zscore strategy

In [323]:
import pandas as pd
import numpy as np

df=pd.read_parquet('data.parquet', engine='pyarrow')
df

,banknifty,nifty,tte
time,,,
2021-01-01 09:15:00,0.286058,0.199729,27
2021-01-01 09:16:00,0.285381,0.200433,27
2021-01-01 09:17:00,0.284233,0.200004,27
2021-01-01 09:18:00,0.286104,0.199860,27
2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...
2022-06-30 15:26:00,0.240701,0.214758,28
2022-06-30 15:27:00,0.240875,0.216558,28
2022-06-30 15:28:00,0.242115,0.216794,28


In [324]:
# this dataframe is to display the final values for both the strategies
df_final=pd.DataFrame(columns=['P/L','sharpe ratio','drawdown'])

#### Removing unnecessary time data and keeping only the data when markets were open

In [325]:
df = df[(df.index.time >= pd.to_datetime('09:15:00').time()) & (df.index.time <=pd.to_datetime('15:30:00').time())]
df

,banknifty,nifty,tte
time,,,
2021-01-01 09:15:00,0.286058,0.199729,27
2021-01-01 09:16:00,0.285381,0.200433,27
2021-01-01 09:17:00,0.284233,0.200004,27
2021-01-01 09:18:00,0.286104,0.199860,27
2021-01-01 09:19:00,0.285539,0.198951,27
...,...,...,...
2022-06-30 15:26:00,0.240701,0.214758,28
2022-06-30 15:27:00,0.240875,0.216558,28
2022-06-30 15:28:00,0.242115,0.216794,28


In [326]:
df_col=df.columns #save the columns as imputer removes them

In [327]:
from sklearn.impute import SimpleImputer

# imputer to replace any Nan or Null values
imputer = SimpleImputer(strategy='mean')

df=pd.DataFrame(imputer.fit_transform(df),columns=df_col)

In [328]:
df['spread']=df['banknifty']-df['nifty']

mean_abs_spread=df['spread'].abs().mean()
std_spread=df['spread'].std()

print(df)
print(f"mean spread: {mean_abs_spread}")
print(f"std spread: {std_spread}")

        banknifty     nifty   tte    spread
0        0.286058  0.199729  27.0  0.086329
1        0.285381  0.200433  27.0  0.084948
2        0.284233  0.200004  27.0  0.084229
3        0.286104  0.199860  27.0  0.086244
4        0.285539  0.198951  27.0  0.086588
...           ...       ...   ...       ...
180851   0.240701  0.214758  28.0  0.025943
180852   0.240875  0.216558  28.0  0.024317
180853   0.242115  0.216794  28.0  0.025321
180854   0.243426  0.216455  28.0  0.026971
180855   0.241907  0.216081  28.0  0.025827

[180856 rows x 4 columns]
mean spread: 0.07191352357336811
std spread: 0.026724355019300296


In [329]:
df['spread'].max()

0.24208459641671146

In [330]:
df['spread'].min()

-0.030397797698990536

In [331]:
df['zscore']=(df['spread'].abs()-mean_abs_spread)/std_spread

#### Choosing thresholds for taking trading decisions

In [332]:
df['trade']=0 # Hold

df.loc[df['zscore']<-2.2,'trade']=1 #Buy niftybank and Sell nifty
df.loc[df['zscore']>2.5,'trade']=-1 #Sell niftybank and Buy nifty

df['entry or exit']='None'
for i in range(len(df.index)-1):
    if df.loc[i,'trade']==0 and df.loc[i+1,'trade']!=0:
        df.loc[i+1,'entry or exit']='entry'
    elif df.loc[i,'trade']!=0 and df.loc[i+1,'trade']==0:
        df.loc[i+1,'entry or exit']='exit'  
    elif df.loc[i,'trade']!=0 and df.loc[i+1,'trade']!=df.loc[i,'trade']:
        df.loc[i+1,'entry or exit']='exit previous and enter new order'   


df

,banknifty,nifty,tte,spread,zscore,trade,entry or exit
0,0.286058,0.199729,27.0,0.086329,0.539413,0,None
1,0.285381,0.200433,27.0,0.084948,0.487756,0,None
2,0.284233,0.200004,27.0,0.084229,0.460833,0,None
3,0.286104,0.199860,27.0,0.086244,0.536214,0,None
4,0.285539,0.198951,27.0,0.086588,0.549105,0,None
...,...,...,...,...,...,...,...
180851,0.240701,0.214758,28.0,0.025943,-1.720173,0,None
180852,0.240875,0.216558,28.0,0.024317,-1.781035,0,None
180853,0.242115,0.216794,28.0,0.025321,-1.743448,0,None
180854,0.243426,0.216455,28.0,0.026971,-1.681707,0,None


In [333]:
df['PbyL']=df['spread']*df['tte']**0.7
df

,banknifty,nifty,tte,spread,zscore,trade,entry or exit,PbyL
0,0.286058,0.199729,27.0,0.086329,0.539413,0,None,0.867184
1,0.285381,0.200433,27.0,0.084948,0.487756,0,None,0.853317
2,0.284233,0.200004,27.0,0.084229,0.460833,0,None,0.846089
3,0.286104,0.199860,27.0,0.086244,0.536214,0,None,0.866325
4,0.285539,0.198951,27.0,0.086588,0.549105,0,None,0.869786
...,...,...,...,...,...,...,...,...
180851,0.240701,0.214758,28.0,0.025943,-1.720173,0,None,0.267320
180852,0.240875,0.216558,28.0,0.024317,-1.781035,0,None,0.250560
180853,0.242115,0.216794,28.0,0.025321,-1.743448,0,None,0.260910
180854,0.243426,0.216455,28.0,0.026971,-1.681707,0,None,0.277912


In [334]:
total_profit=0
profits_arr=[]  #array with profits for all the trades we take
entry_arr=[]
exit_arr=[]
pbyl=[]

for i,e in enumerate(df['entry or exit']):
    if e=='entry':
        entry_arr.append(i)
    if e=='exit':
        exit_arr.append(i)

if len(entry_arr)>len(exit_arr):
    entry_arr=entry_arr[:-1]

for x,y in zip(entry_arr,exit_arr):
    pbyl.append(df.loc[x,'spread']*df.loc[x,'tte']**0.7) #assuming the PbyL here as this
    profit=df.loc[x,'trade']*(df.loc[y,'spread']-df.loc[x,'spread'])  # when we buy nifty bank and sell nifty, we will get a profit if the spread increases(Here I am assuming the change in the spread as the profit)
    # the multiplication with the 'trade' signal ensures that this works for the trade of selling niftybank and buying nifty
    profits_arr.append(profit)
    total_profit+=profit

profits_arr=np.array(profits_arr)
mean_profit=pd.Series(profits_arr).mean()
std_profit=pd.Series(profits_arr).std()
sharpe_ratio=mean_profit/std_profit


print(f'P/L: {np.array(pbyl).sum()}') # Assuming the total PbyL as the sum of all the PbyLs
print(f'sharpe ratio: {sharpe_ratio}')
print(f'drawdown: {-1*profits_arr.min()}')

df_final.loc['Zscore','P/L']=np.array(pbyl).sum()
df_final.loc['Zscore','sharpe ratio']=sharpe_ratio
df_final.loc['Zscore','drawdown']=-1*profits_arr.min()



P/L: 356.0652029683129
sharpe ratio: 1.4280915504349028
drawdown: -0.00019849999999999035


# My Strategy

In [335]:
df=df.iloc[:,:4] #to remove all the extra columns we used in the last strategy
df

,banknifty,nifty,tte,spread
0,0.286058,0.199729,27.0,0.086329
1,0.285381,0.200433,27.0,0.084948
2,0.284233,0.200004,27.0,0.084229
3,0.286104,0.199860,27.0,0.086244
4,0.285539,0.198951,27.0,0.086588
...,...,...,...,...
180851,0.240701,0.214758,28.0,0.025943
180852,0.240875,0.216558,28.0,0.024317
180853,0.242115,0.216794,28.0,0.025321
180854,0.243426,0.216455,28.0,0.026971


In [336]:
df['bollinger_mean']=df['spread'].rolling(window=20).mean()
df['bollinger_std']=df['spread'].rolling(window=20).std()
df=df[20:].reset_index(drop=True) # to avoid the nan values at the start
df['bollinger_upper']=df['bollinger_mean']+4*df['bollinger_std']
df['bollinger_lower']=df['bollinger_mean']-4*df['bollinger_std']

df['trade'] =0
df.loc[df['spread']<df['bollinger_lower'],'trade']=1 #Buy niftybank and Sell nifty
df.loc[df['spread']>df['bollinger_upper'],'trade']=-1 #Sell niftybank and Buy nifty

df['entry or exit']='None'
for i in range(len(df.index)-1):
    if df.loc[i,'trade']==0 and df.loc[i+1,'trade']!=0:
        df.loc[i+1,'entry or exit']='entry'
    elif df.loc[i,'trade']!=0 and df.loc[i+1,'trade']==0:
        df.loc[i+1,'entry or exit']='exit'  
    elif df.loc[i,'trade']!=0 and df.loc[i+1,'trade']!=df.loc[i,'trade']:
        df.loc[i+1,'entry or exit']='exit previous and enter new order'   


df

,banknifty,nifty,tte,spread,bollinger_mean,bollinger_std,bollinger_upper,bollinger_lower,trade,entry or exit
0,0.276839,0.194429,27.0,0.082410,0.084016,0.001769,0.091091,0.076941,0,None
1,0.275827,0.194350,27.0,0.081477,0.083842,0.001841,0.091207,0.076477,0,None
2,0.275327,0.195977,27.0,0.079349,0.083598,0.002093,0.091971,0.075225,0,None
3,0.274066,0.194173,27.0,0.079893,0.083281,0.002152,0.091888,0.074673,0,None
4,0.273338,0.193976,27.0,0.079362,0.082919,0.002174,0.091614,0.074224,0,None
...,...,...,...,...,...,...,...,...,...,...
180831,0.240701,0.214758,28.0,0.025943,0.025707,0.001000,0.029708,0.021705,0,None
180832,0.240875,0.216558,28.0,0.024317,0.025581,0.001010,0.029619,0.021543,0,None
180833,0.242115,0.216794,28.0,0.025321,0.025509,0.000971,0.029393,0.021624,0,None
180834,0.243426,0.216455,28.0,0.026971,0.025510,0.000973,0.029403,0.021617,0,None


In [337]:
len(df[df['trade']==1].index)

330

In [338]:
total_profit=0
profits_arr=[]  #array with profits for all the trades we take
entry_arr=[]
exit_arr=[]
pbyl=[]

for i,e in enumerate(df['entry or exit']):
    if e=='entry':
        entry_arr.append(i)
    if e=='exit':
        exit_arr.append(i)

if len(entry_arr)>len(exit_arr):
    entry_arr=entry_arr[:-1]

for x,y in zip(entry_arr,exit_arr):
    pbyl.append(df.loc[x,'spread']*df.loc[x,'tte']**0.7) #assuming the PbyL here as this
    profit=df.loc[x,'trade']*(df.loc[y,'spread']-df.loc[x,'spread'])  # when we buy nifty bank and sell nifty, we will get a profit if the spread increases(Here I am assuming the change in the spread as the profit)
    # the multiplication with the 'trade' signal ensures that this works for the trade of selling niftybank and buying nifty
    profits_arr.append(profit)
    total_profit+=profit

profits_arr=np.array(profits_arr)
mean_profit=pd.Series(profits_arr).mean()
std_profit=pd.Series(profits_arr).std()
sharpe_ratio=mean_profit/std_profit


print(f'P/L: {np.array(pbyl).sum()}') # Assuming the total PbyL as the sum of all the PbyLs
print(f'sharpe ratio: {sharpe_ratio}')
print(f'drawdown: {-1*profits_arr.min()}')

df_final.loc['My strategy','P/L']=np.array(pbyl).sum()
df_final.loc['My strategy','sharpe ratio']=sharpe_ratio
df_final.loc['My strategy','drawdown']=-1*profits_arr.min()


P/L: 386.07642999888037
sharpe ratio: 1.432826147119089
drawdown: 0.017448499999999978


In [339]:
df_final

,P/L,sharpe ratio,drawdown
Zscore,356.065203,1.428092,-0.000198
My strategy,386.07643,1.432826,0.017448


#### As we can see, my strategy has a higher P/L and also a slightly better sharpe ratio than the zscore model. But the drawdown however is smaller for zscore strategy and it is actually negative i.e. no trade taken had a loss